In [ ]:
import pandas as pd
import operator
import time
import numpy as np

In [ ]:
config = {}

config["project_path"] = ".."
config["data_folder_path"] = "regional_data"
config["temp_folder_path"] = "regional_temp"

# input files
config["SFA_file_pattern"] = '中台rawdata-易掌柜数据源-{}月.xlsx'
config["SOS_file"] = 'SOS.csv'
config["Mustsell_file"] = '必分销率.csv'
config["customer_input_file"] = ["2020 MDM.xlsx"]
config["grade_channel_file"] = "grade_channel.xlsx"
config["customer_acct_file"] = "customer_code_account_name.xlsx"
config["sellout_input_files"] = ['DW_Fact_Sellout_NoneSplit_202001-03.csv', 'DW_Fact_Sellout_NoneSplit_202004-05.csv', 'DW_Fact_Sellout_NoneSplit_202006.csv']
config["product_file"] = "DW_DIM_Product_20200609.csv"

# output files
config["SOS_geo_file"] = 'SOS_geo_grade_2020.xlsx'
config["OOS_grade_file"] = "OOS_grade_202006_YTD.xlsx"
config["SOS_geo_wt_file"] = 'SOS_geo_grade_wt_2020.xlsx'
config["OOS_grade_wt_file"] = "OOS_grade_wt_202006_YTD.xlsx"
config["dist_geo_file"] = 'distribution_geo_grade_2020.xlsx'
config["dist_geo_wt_file"] = 'distribution_geo_grade_wt_2020.xlsx'
config["mustsell_geo_file"] = "mustsell_geo_202006_YTD.xlsx"
config["layout_geo_file"] = "layout_geo_202006_YTD.xlsx"
config["layout_geo_wt_file"] = "layout_geo_wt_202006_YTD.xlsx"

# Read Input

In [ ]:
def remove_starting_zero(customer_code):
    if customer_code.startswith("0"):
        return customer_code[1:]
    return customer_code

## V_Report_SFA_Inventory_By_SKU

In [ ]:
sfa_columns = ['Year Month', 'Region Name', 'Subregion Name', 'Customer Code', 'Account Name', 'Grade Code',
              'Sku', 'Distributed Store', 'Layout Store', 'Shortage Store']
sfa_input_df = pd.DataFrame(columns=sfa_columns)

In [ ]:
for month in range(1, 7):
    year_month = str(2020) + str(month).zfill(2)
    input_file_path = config["project_path"] + "/" \
                        + config["data_folder_path"] + "/" \
                        + config["SFA_file_pattern"].format(month)

    input_df = pd.read_excel(input_file_path, dtype=str)[sfa_columns]
    sfa_input_df = sfa_input_df.append(input_df)

In [ ]:
sfa_input_df.columns = ['YearMonth', 'Region_Name', 'Subregion', 'Customer_Code', 'Account_Name', 'Grade_Code',
              'SKU', 'Distributed_Store', 'Layout_Store', 'Shortage_Store']

In [ ]:
sfa_input_df = sfa_input_df.rename(columns={'Sku': 'SKU'})
sfa_input_df["Layout_Store"] = sfa_input_df["Layout_Store"].astype("float")
sfa_input_df["Shortage_Store"] = sfa_input_df["Shortage_Store"].astype("float")
sfa_input_df["Distributed_Store"] = sfa_input_df["Distributed_Store"].astype("float")
sfa_input_df["Customer_Code"] = sfa_input_df["Customer_Code"].astype("str")
sfa_input_df["Customer_Code"] = sfa_input_df['Customer_Code'].apply(lambda x:remove_starting_zero(x))
sfa_input_df["Customer_Code"] = sfa_input_df['Customer_Code'].apply(lambda x:remove_starting_zero(x))

In [ ]:
input_file_path = config["project_path"] + "/" \
                        + config["data_folder_path"] + "/" \
                        + 'sfa_input.csv'

sfa_input_df.to_csv(input_file_path, index=False)

## V_Report_SFA_SOS

In [ ]:
input_file_path = config["project_path"] + "/" \
                        + config["data_folder_path"] + "/" \
                        + config["SOS_file"]

SOS_df = pd.read_csv(input_file_path, dtype=str,sep = '\t')
SOS_df.columns = ['YearMonth', 'Customer_Code', 'All_Group', 'Brand_Group', 'SOS']

SOS_df["All_Group"] = SOS_df["All_Group"].astype("float")
SOS_df["Brand_Group"] = SOS_df["Brand_Group"].astype("float")
SOS_df["SOS"] = SOS_df["SOS"].astype("float")
SOS_df["Customer_Code"] = SOS_df["Customer_Code"].astype("str")
SOS_df["Customer_Code"] = SOS_df['Customer_Code'].apply(lambda x:remove_starting_zero(x))
SOS_df["Customer_Code"] = SOS_df['Customer_Code'].apply(lambda x:remove_starting_zero(x))

## V_Report_SFA_Inventory_By_Customer

In [ ]:
input_file_path = config["project_path"] + "/" \
                        + config["data_folder_path"] + "/" \
                        + config["Mustsell_file"]

must_sell_df = pd.read_csv(input_file_path, dtype=str,sep = '\t')
must_sell_df.columns = ['YearMonth', 'Customer_Code', 'Mustsell_All_Distributed', 'Mustsell_Stock_ Available', 'Mustsell_Stock_Shortage']

must_sell_df["Mustsell_All_Distributed"] = must_sell_df["Mustsell_All_Distributed"].astype("float")
must_sell_df["Mustsell_Stock_ Available"] = must_sell_df["Mustsell_Stock_ Available"].astype("float")
must_sell_df["Mustsell_Stock_Shortage"] = must_sell_df["Mustsell_Stock_Shortage"].astype("float")
must_sell_df["Customer_Code"] = must_sell_df['Customer_Code'].apply(lambda x:remove_starting_zero(x))
must_sell_df["Customer_Code"] = must_sell_df['Customer_Code'].apply(lambda x:remove_starting_zero(x))

## Customer

In [ ]:
# Read customer information

select_columns = ['YearMonth','Customer_Code','Grade_Code',
                  'region_name','subregion_name', 'province_name']

customer_input_df = pd.DataFrame(columns=select_columns)

In [ ]:
for customer_input_file in config["customer_input_file"]:
    input_file_path = config["project_path"] + "/" \
                        + config["data_folder_path"] + "/" \
                        + customer_input_file
    input_df = pd.read_excel(input_file_path, dtype=str)[select_columns]
    customer_input_df = customer_input_df.append(input_df)

## Grade Code Mapping

In [ ]:
input_file_path = config["project_path"] + "/" \
                    + config["data_folder_path"] + "/" \
                    + config["grade_channel_file"] 

grade_df = pd.read_excel(input_file_path, dtype=str)

In [ ]:
cust_grade_mapping = pd.merge(customer_input_df, grade_df, left_on='Grade_Code', right_on='Grade', how='left')

input_file_path = config["project_path"] + "/" \
                        + config["data_folder_path"] + "/" \
                        + 'cust_grade_mapping.csv'

cust_grade_mapping = cust_grade_mapping.to_csv(input_file_path, index=False)

## Account name

In [ ]:
input_file_path = config["project_path"] + "/" \
                    + config["data_folder_path"] + "/" \
                    + config["customer_acct_file"] 

acct_df = pd.read_excel(input_file_path, dtype=str)
acct_df['Customer_Code'] = acct_df['Customer_Code'].apply(lambda x:remove_starting_zero(x))
acct_df['Customer_Code'] = acct_df['Customer_Code'].apply(lambda x:remove_starting_zero(x))

## Sellout

In [ ]:
# schema of input file
sellout_columns = ['YearMonth', 'Data_Source', 'Bill_Date', 'Bill_Type', 
                   'Distributor_Code_DMS', 'Distributor_Code', 'Client_Code', 
                   'Client_Name', 'EA_Platform', 'Ship_To_Code', 
                   'Inventory_location', 'Customer_Code', 'Product_Code', 
                   'Sellout_Qty', 'Sellout_SP_Value', 'Sellout_Channel_Value', 
                   'ETL_DateTime', 'Data_CreationTime']

# Only below columns from input file are needed
select_columns = ['YearMonth','Customer_Code',
                  'Product_Code','Sellout_SP_Value']
sellout_df = pd.DataFrame(columns=select_columns)

In [ ]:
for sellout_input_file in config["sellout_input_files"]:
    input_file_path = config["project_path"] + "/" \
                        + config["data_folder_path"] + "/" \
                        + sellout_input_file
    input_df = pd.read_csv(input_file_path, header = None, dtype=str).fillna('0')
    input_df.columns = sellout_columns
    if operator.eq(list(input_df.loc[0]),list(sellout_columns))== True:
        input_df = input_df.drop([0])
    input_df = input_df[select_columns]
    sellout_df = sellout_df.append(input_df, ignore_index=True)

In [ ]:
sellout_df["Sellout_SP_Value"] = sellout_df["Sellout_SP_Value"].astype("float")
sellout_df["Product_Code"] = sellout_df["Product_Code"].astype("str")
sellout_df["Customer_Code"] = sellout_df["Customer_Code"].astype("str")
sellout_df["YearMonth"] = sellout_df["YearMonth"].astype("str")

sellout_df["Customer_Code"] = sellout_df['Customer_Code'].apply(lambda x:remove_starting_zero(x))
sellout_df["Customer_Code"] = sellout_df['Customer_Code'].apply(lambda x:remove_starting_zero(x))

## Product mapping

In [ ]:
product_brand_columns = ["Product_Code", "Product_Short_NameEN"]

input_file_path = config["project_path"] + "/" + config["data_folder_path"] + "/" + config["product_file"]

product_brand_input_df = pd.read_csv(input_file_path, header=0, dtype=str)[product_brand_columns]

product_brand_output_df = product_brand_input_df.drop_duplicates().reset_index(drop=True)

product_brand_output_df.columns = ["Product_Code", "Brand_Stage"]

product_brand_output_df = product_brand_output_df.dropna()

product_brand_output_df["Brand"] = product_brand_output_df["Brand_Stage"].str.slice(0, 2)

# Add sepcial product code for brands and all brands
product_brand_output_df = product_brand_output_df.append(
                        [{"Product_Code":"10332781", "Brand_Stage": "NC", "Brand": "NC"}], ignore_index=True)

product_brand_output_df = product_brand_output_df.append(
                        [{"Product_Code":"10332872", "Brand_Stage": "AC", "Brand": "AC"}], ignore_index=True)

product_brand_output_df = product_brand_output_df.append(
                        [{"Product_Code":"10332873", "Brand_Stage": "AP", "Brand": "AP"}], ignore_index=True)

product_brand_output_df = product_brand_output_df.append(
                        [{"Product_Code":"10332778", "Brand_Stage": "ELN", "Brand": "ELN"}], ignore_index=True)

In [ ]:
sellout_with_brand = pd.merge(sellout_df, product_brand_output_df, on="Product_Code", how="left")
sellout_with_brand = sellout_with_brand[sellout_with_brand.Brand.isin(["AC", "NC", "AP"])]

In [ ]:
def get_Channel(Grade_Code):
    if Grade_Code=='RKMBS':
        return 'RKMBS'
    elif (Grade_Code=='NKA') | (Grade_Code=='RKA') | (Grade_Code=='LKA') | (Grade_Code=='NKMBS'):
        return 'KA'
    else:
        return 'Others'
    
def get_past_3m(yearmonth):
    yearmonth_int = int(yearmonth)
    p3m_list = []
    for i in range(3):
        yearmonth_int = yearmonth_int - 1
        if yearmonth_int%100 == 0:
            yearmonth_int = yearmonth_int - 100
            yearmonth_int = yearmonth_int + 12
        p3m_list = p3m_list + [str(yearmonth_int)]
    return p3m_list

# SOS

In [ ]:
SOS_grade_df = pd.merge(SOS_df, cust_grade_mapping, on=['YearMonth', 'Customer_Code'], how='left')
SOS_grade_df = SOS_grade_df.loc[SOS_grade_df['YearMonth'].isin(['202001', '202002', '202003', '202004', '202005', '202006'])].copy()
SOS_grade_df['Channel'] = SOS_grade_df['Channel'].apply(lambda x:get_Channel(x))

In [ ]:
SOS_grade_df = SOS_grade_df.loc[SOS_grade_df['Channel']!='Others']

In [ ]:
sos_cal_columns = ['YearMonth', 'Geo_Name', 'region_name', 'subregion_name', 'ASM', 'Channel', 'All_Group', 'Brand_Group']

sos_cal_df = pd.DataFrame(columns=sos_cal_columns)

In [ ]:
sos_temp = SOS_grade_df.groupby(by = ['YearMonth', 'Channel'])[['All_Group', 'Brand_Group']].sum().reset_index()
sos_temp['Geo_Name'] = '全国'
sos_temp['region_name'] = ''
sos_temp['subregion_name'] = ''
sos_temp['ASM'] = ''
sos_cal_df = sos_cal_df.append(sos_temp)

sos_temp = SOS_grade_df.groupby(by = ['YearMonth', 'Channel', 'region_name'])[['All_Group', 'Brand_Group']].sum().reset_index()
sos_temp['Geo_Name'] = sos_temp['region_name']
sos_temp['subregion_name'] = ''
sos_temp['ASM'] = ''
sos_cal_df = sos_cal_df.append(sos_temp)

sos_temp = SOS_grade_df.groupby(by = ['YearMonth', 'Channel', 'region_name', 'subregion_name'])[['All_Group', 'Brand_Group']].sum().reset_index()
sos_temp['Geo_Name'] = sos_temp['subregion_name']
sos_temp['ASM'] = ''
sos_cal_df = sos_cal_df.append(sos_temp)

sos_cal_df = sos_cal_df.rename(columns={'region_name': 'Region_Name',
                                        'subregion_name': 'Subregion',
                                        'All_Group': 'All Group',
                                        'Brand_Group': 'Brand Group'})

In [ ]:
sos_cal_columns = ['YearMonth', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel', 'All Group', 'Brand Group']

p3m_df = pd.DataFrame(columns=sos_cal_columns)

In [ ]:
year = 2020
for month in range(5, 7):
    year_month = str(year) + str(month).zfill(2)
    p3m_list = get_past_3m(year_month)
    
    p3m_sos = sos_cal_df.loc[sos_cal_df['YearMonth'].isin(p3m_list)].copy()
    sos_temp = p3m_sos.groupby(by = ['Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel'])[['All Group', 'Brand Group']].mean().reset_index()
    sos_temp['YearMonth'] = year_month
    
    p3m_df = p3m_df.append(sos_temp)
    
p3m_df = p3m_df.rename(columns={'All Group': 'P3M All Group', 'Brand Group': 'P3M Brand Group'})

In [ ]:
sos_res_df = pd.merge(sos_cal_df, p3m_df, on=['YearMonth', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel'])

In [ ]:
output_file_path = config["project_path"] + "/" \
                    + config["temp_folder_path"] + "/" \
                    + config["SOS_geo_file"]

sos_res_df.to_excel(output_file_path, header=True, index=False, float_format="%.2f")

# SOS Weighted

In [ ]:
SOS_grade_df = pd.merge(SOS_df, cust_grade_mapping, on=['YearMonth', 'Customer_Code'], how='left')
SOS_grade_df = SOS_grade_df.loc[SOS_grade_df['YearMonth'].isin(['202001', '202002', '202003', '202004', '202005', '202006'])].copy()
SOS_grade_df['Channel'] = SOS_grade_df['Channel'].apply(lambda x:get_Channel(x))

sellout_cust = sellout_with_brand.groupby(by = ['YearMonth', 'Customer_Code'])[['Sellout_SP_Value']].sum().reset_index()

SOS_sellout_df = pd.merge(SOS_grade_df, sellout_cust, on=['YearMonth', 'Customer_Code'], how='left')
SOS_sellout_df = SOS_sellout_df.fillna({'Sellout_SP_Value':0})

SOS_sellout_df['All_Group'] = SOS_sellout_df['All_Group'] * SOS_sellout_df['Sellout_SP_Value']
SOS_sellout_df['Brand_Group'] = SOS_sellout_df['Brand_Group'] * SOS_sellout_df['Sellout_SP_Value']

SOS_sellout_df = SOS_sellout_df.loc[SOS_sellout_df['Channel']!='Others']

In [ ]:
sos_cal_columns = ['YearMonth', 'Geo_Name', 'region_name', 'subregion_name', 'ASM', 'Channel', 'All_Group', 'Brand_Group']

sos_cal_df = pd.DataFrame(columns=sos_cal_columns)

sos_temp = SOS_sellout_df.groupby(by = ['YearMonth', 'Channel'])[['All_Group', 'Brand_Group']].sum().reset_index()
sos_temp['Geo_Name'] = '全国'
sos_temp['region_name'] = ''
sos_temp['subregion_name'] = ''
sos_temp['ASM'] = ''
sos_cal_df = sos_cal_df.append(sos_temp)

sos_temp = SOS_sellout_df.groupby(by = ['YearMonth', 'Channel', 'region_name'])[['All_Group', 'Brand_Group']].sum().reset_index()
sos_temp['Geo_Name'] = sos_temp['region_name']
sos_temp['subregion_name'] = ''
sos_temp['ASM'] = ''
sos_cal_df = sos_cal_df.append(sos_temp)

sos_temp = SOS_sellout_df.groupby(by = ['YearMonth', 'Channel', 'region_name', 'subregion_name'])[['All_Group', 'Brand_Group']].sum().reset_index()
sos_temp['Geo_Name'] = sos_temp['subregion_name']
sos_temp['ASM'] = ''
sos_cal_df = sos_cal_df.append(sos_temp)

sos_cal_df = sos_cal_df.rename(columns={'region_name': 'Region_Name',
                                        'subregion_name': 'Subregion',
                                        'All_Group': 'Weighted All Group',
                                        'Brand_Group': 'Weighted Brand Group'})

In [ ]:
sos_cal_columns = ['YearMonth', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel', 'Weighted All Group', 'Weighted Brand Group']

p3m_df = pd.DataFrame(columns=sos_cal_columns)

year = 2020
for month in range(5, 7):
    year_month = str(year) + str(month).zfill(2)
    p3m_list = get_past_3m(year_month)
    
    p3m_sos = sos_cal_df.loc[sos_cal_df['YearMonth'].isin(p3m_list)].copy()
    sos_temp = p3m_sos.groupby(by = ['Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel'])[['Weighted All Group', 'Weighted Brand Group']].mean().reset_index()
    sos_temp['YearMonth'] = year_month
    
    p3m_df = p3m_df.append(sos_temp)
    
p3m_df = p3m_df.rename(columns={'Weighted All Group': 'P3M Weighted All Group', 'Weighted Brand Group': 'P3M Weighted Brand Group'})

In [ ]:
sos_res_df = pd.merge(sos_cal_df, p3m_df, on=['YearMonth', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel'])

output_file_path = config["project_path"] + "/" \
                    + config["temp_folder_path"] + "/" \
                    + config["SOS_geo_wt_file"]

sos_res_df.to_excel(output_file_path, header=True, index=False, float_format="%.2f")

# Distribution

In [ ]:
def get_brand(SKU):
    if SKU.startswith('AP'):
        return 'AP'
    elif SKU.startswith('AC'):
        return 'AC'
    elif SKU.startswith('NC'):
        return 'NC'
    return SKU


def get_Grade_Group(Grade_Code):
    if  (Grade_Code=='RKMBS-G') | (Grade_Code=='RKMBS-M') | (Grade_Code=='RKMBS'):
        return 'RKMBS'
    elif  (Grade_Code=='NKMBS-G') | (Grade_Code=='NKMBS'):
        return 'NKMBS'
    elif (Grade_Code=='NKA') | (Grade_Code=='RKA') | (Grade_Code=='LKA'):
        return 'MT'
    else:
        return 'Others'

In [ ]:
sfa_cal_df = sfa_input_df
sfa_cal_df['Channel'] = sfa_cal_df['Grade_Code'].apply(lambda x:get_Channel(x))
sfa_cal_df["Brand"] = sfa_cal_df['SKU'].apply(lambda x:get_brand(x))

sfa_cal_df['Grade_Group'] = sfa_cal_df['Grade_Code'].apply(lambda x:get_Grade_Group(x))

In [ ]:
dist_df = sfa_cal_df.loc[sfa_cal_df['Channel']!='Others']

In [ ]:
dist_cal_columns = ['YearMonth', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel', 'Brand', 'Customer_Code', 'Distributed_Store']
dist_cal_df = pd.DataFrame(columns=dist_cal_columns)

dist_temp = dist_df.groupby(by = ['YearMonth', 'Channel'])['Distributed_Store'].sum().reset_index()
dist_temp['Geo_Name'] = '全国'
dist_temp['Region_Name'] = ''
dist_temp['Subregion'] = ' '
dist_temp['ASM'] = ''

ttl_temp = dist_df.groupby(by = ['YearMonth', 'Channel'])['Customer_Code'].count().reset_index()
ttl_temp['Geo_Name'] = '全国'
ttl_temp['Region_Name'] = ''
ttl_temp['Subregion'] = ' '
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, dist_temp, on=['YearMonth', 'Channel', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
df_temp['Brand'] = 'TTL'
dist_cal_df = dist_cal_df.append(df_temp)


dist_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Region_Name'])['Distributed_Store'].sum().reset_index()
dist_temp['Geo_Name'] = dist_temp['Region_Name']
dist_temp['Subregion'] = ' '
dist_temp['ASM'] = ''

ttl_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Region_Name'])['Customer_Code'].count().reset_index()
ttl_temp['Geo_Name'] = ttl_temp['Region_Name']
ttl_temp['Subregion'] = ' '
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, dist_temp, on=['YearMonth', 'Channel', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
df_temp['Brand'] = 'TTL'
dist_cal_df = dist_cal_df.append(df_temp)


dist_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Region_Name', 'Subregion'])['Distributed_Store'].sum().reset_index()
dist_temp['Geo_Name'] = dist_temp['Subregion']
dist_temp['ASM'] = ''

ttl_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Region_Name', 'Subregion'])['Customer_Code'].count().reset_index()
ttl_temp['Geo_Name'] = ttl_temp['Subregion']
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, dist_temp, on=['YearMonth', 'Channel', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
df_temp['Brand'] = 'TTL'
dist_cal_df = dist_cal_df.append(df_temp)

# By Brand
dist_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Brand'])['Distributed_Store'].sum().reset_index()
dist_temp['Geo_Name'] = '全国'
dist_temp['Region_Name'] = ''
dist_temp['Subregion'] = ' '
dist_temp['ASM'] = ''

ttl_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Brand'])['Customer_Code'].count().reset_index()
ttl_temp['Geo_Name'] = '全国'
ttl_temp['Region_Name'] = ''
ttl_temp['Subregion'] = ' '
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, dist_temp, on=['YearMonth', 'Channel', 'Brand', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
dist_cal_df = dist_cal_df.append(df_temp)


dist_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Brand', 'Region_Name'])['Distributed_Store'].sum().reset_index()
dist_temp['Geo_Name'] = dist_temp['Region_Name']
dist_temp['Subregion'] = ' '
dist_temp['ASM'] = ''

ttl_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Brand', 'Region_Name'])['Customer_Code'].count().reset_index()
ttl_temp['Geo_Name'] = ttl_temp['Region_Name']
ttl_temp['Subregion'] = ' '
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, dist_temp, on=['YearMonth', 'Channel', 'Brand', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
dist_cal_df = dist_cal_df.append(df_temp)


dist_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Brand', 'Region_Name', 'Subregion'])['Distributed_Store'].sum().reset_index()
dist_temp['Geo_Name'] = dist_temp['Subregion']
dist_temp['ASM'] = ''

ttl_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Brand', 'Region_Name', 'Subregion'])['Customer_Code'].count().reset_index()
ttl_temp['Geo_Name'] = ttl_temp['Subregion']
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, dist_temp, on=['YearMonth', 'Channel', 'Brand', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
dist_cal_df = dist_cal_df.append(df_temp)

dist_cal_df["Customer_Code"] = dist_cal_df["Customer_Code"].astype("float") 
dist_cal_df = dist_cal_df.rename(columns={'Customer_Code' : 'Require Distribute Stores',
                                            'Distributed_Store' : 'Distributed Stores'})

In [ ]:
dist_cal_columns = ['YearMonth', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel', 'Brand', 'Require Distribute Stores', 'Distributed Stores']

p3m_df = pd.DataFrame(columns=dist_cal_columns)

year = 2020
for month in range(5, 7):
    year_month = str(year) + str(month).zfill(2)
    p3m_list = get_past_3m(year_month)
    
    p3m_dist = dist_cal_df.loc[dist_cal_df['YearMonth'].isin(p3m_list)].copy()
    dist_temp = p3m_dist.groupby(by = ['Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel', 'Brand'])[['Require Distribute Stores', 'Distributed Stores']].mean().reset_index()
    dist_temp['YearMonth'] = year_month
    
    p3m_df = p3m_df.append(dist_temp)
    
p3m_df['Require Distribute Stores'] = p3m_df['Require Distribute Stores'].round(0)
p3m_df['Distributed Stores'] = p3m_df['Distributed Stores'].round(0)

In [ ]:
p3m_df = p3m_df.rename(columns={'Require Distribute Stores': 'P3M Require Distribute', 'Distributed Stores': 'P3M Distributed Stores'})

In [ ]:
dist_res_df = pd.merge(dist_cal_df, p3m_df, on=['YearMonth', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel', 'Brand'])

output_file_path = config["project_path"] + "/" \
                    + config["temp_folder_path"] + "/" \
                    + config["dist_geo_file"]

dist_res_df.to_excel(output_file_path, header=True, index=False, float_format="%.2f")

# Distribution Weighted

In [ ]:
sfa_cal_df = sfa_input_df
sfa_cal_df['Channel'] = sfa_cal_df['Grade_Code'].apply(lambda x:get_Channel(x))
sfa_cal_df["Brand"] = sfa_cal_df['SKU'].apply(lambda x:get_brand(x))

sfa_cal_df['Grade_Group'] = sfa_cal_df['Grade_Code'].apply(lambda x:get_Grade_Group(x))


sellout_cust_sku = sellout_with_brand.groupby(by = ['YearMonth', 'Customer_Code', 'Brand_Stage'])[['Sellout_SP_Value']].sum().reset_index()
sellout_cust_sku.columns = ['YearMonth', 'Customer_Code', 'SKU', 'Sellout_SP_Value']

sfa_cal_df = pd.merge(sfa_cal_df, sellout_cust_sku, on=['YearMonth', 'Customer_Code', 'SKU'], how='left')
sfa_cal_df = sfa_cal_df.fillna({'Sellout_SP_Value':0})

sfa_cal_df['Distributed_Store'] = sfa_cal_df['Distributed_Store'] * sfa_cal_df['Sellout_SP_Value']

In [ ]:
dist_df = sfa_cal_df.loc[sfa_cal_df['Channel']!='Others']

In [ ]:
dist_cal_columns = ['YearMonth', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel', 'Brand', 'Sellout_SP_Value', 'Distributed_Store']
dist_cal_df = pd.DataFrame(columns=dist_cal_columns)

# All Brand
dist_temp = dist_df.groupby(by = ['YearMonth', 'Channel'])['Distributed_Store'].sum().reset_index()
dist_temp['Geo_Name'] = '全国'
dist_temp['Region_Name'] = ''
dist_temp['Subregion'] = ' '
dist_temp['ASM'] = ''

ttl_temp = dist_df.groupby(by = ['YearMonth', 'Channel'])['Sellout_SP_Value'].sum().reset_index()
ttl_temp['Geo_Name'] = '全国'
ttl_temp['Region_Name'] = ''
ttl_temp['Subregion'] = ' '
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, dist_temp, on=['YearMonth', 'Channel', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
df_temp['Brand'] = 'TTL'
dist_cal_df = dist_cal_df.append(df_temp)


dist_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Region_Name'])['Distributed_Store'].sum().reset_index()
dist_temp['Geo_Name'] = dist_temp['Region_Name']
dist_temp['Subregion'] = ' '
dist_temp['ASM'] = ''

ttl_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Region_Name'])['Sellout_SP_Value'].sum().reset_index()
ttl_temp['Geo_Name'] = ttl_temp['Region_Name']
ttl_temp['Subregion'] = ' '
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, dist_temp, on=['YearMonth', 'Channel', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
df_temp['Brand'] = 'TTL'
dist_cal_df = dist_cal_df.append(df_temp)


dist_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Region_Name', 'Subregion'])['Distributed_Store'].sum().reset_index()
dist_temp['Geo_Name'] = dist_temp['Subregion']
dist_temp['ASM'] = ''

ttl_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Region_Name', 'Subregion'])['Sellout_SP_Value'].sum().reset_index()
ttl_temp['Geo_Name'] = ttl_temp['Subregion']
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, dist_temp, on=['YearMonth', 'Channel', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
df_temp['Brand'] = 'TTL'
dist_cal_df = dist_cal_df.append(df_temp)

# By Brand
dist_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Brand'])['Distributed_Store'].sum().reset_index()
dist_temp['Geo_Name'] = '全国'
dist_temp['Region_Name'] = ''
dist_temp['Subregion'] = ' '
dist_temp['ASM'] = ''

ttl_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Brand'])['Sellout_SP_Value'].sum().reset_index()
ttl_temp['Geo_Name'] = '全国'
ttl_temp['Region_Name'] = ''
ttl_temp['Subregion'] = ' '
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, dist_temp, on=['YearMonth', 'Channel', 'Brand', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
dist_cal_df = dist_cal_df.append(df_temp)


dist_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Brand', 'Region_Name'])['Distributed_Store'].sum().reset_index()
dist_temp['Geo_Name'] = dist_temp['Region_Name']
dist_temp['Subregion'] = ' '
dist_temp['ASM'] = ''

ttl_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Brand', 'Region_Name'])['Sellout_SP_Value'].sum().reset_index()
ttl_temp['Geo_Name'] = ttl_temp['Region_Name']
ttl_temp['Subregion'] = ' '
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, dist_temp, on=['YearMonth', 'Channel', 'Brand', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
dist_cal_df = dist_cal_df.append(df_temp)


dist_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Brand', 'Region_Name', 'Subregion'])['Distributed_Store'].sum().reset_index()
dist_temp['Geo_Name'] = dist_temp['Subregion']
dist_temp['ASM'] = ''

ttl_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'Brand', 'Region_Name', 'Subregion'])['Sellout_SP_Value'].sum().reset_index()
ttl_temp['Geo_Name'] = ttl_temp['Subregion']
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, dist_temp, on=['YearMonth', 'Channel', 'Brand', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
dist_cal_df = dist_cal_df.append(df_temp)


dist_cal_df = dist_cal_df.rename(columns={'Sellout_SP_Value' : 'Weighted Require Distribute',
                                            'Distributed_Store' : 'Weighted Distributed Stores'})

In [ ]:
dist_cal_columns = ['YearMonth', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel', 'Brand', 'Weighted Require Distribute', 'Weighted Distributed Stores']

p3m_df = pd.DataFrame(columns=dist_cal_columns)

year = 2020
for month in range(5, 7):
    year_month = str(year) + str(month).zfill(2)
    p3m_list = get_past_3m(year_month)
    
    p3m_dist = dist_cal_df.loc[dist_cal_df['YearMonth'].isin(p3m_list)].copy()
    dist_temp = p3m_dist.groupby(by = ['Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel', 'Brand'])[['Weighted Require Distribute', 'Weighted Distributed Stores']].mean().reset_index()
    dist_temp['YearMonth'] = year_month
    
    p3m_df = p3m_df.append(dist_temp)
    
p3m_df['Weighted Require Distribute'] = p3m_df['Weighted Require Distribute'].round(0)
p3m_df['Weighted Distributed Stores'] = p3m_df['Weighted Distributed Stores'].round(0)

In [ ]:
p3m_df = p3m_df.rename(columns={'Weighted Require Distribute': 'P3M Weighted Require Distribute', 'Weighted Distributed Stores': 'P3M Weighted Distributed'})

In [ ]:
dist_res_df = pd.merge(dist_cal_df, p3m_df, on=['YearMonth', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel', 'Brand'])

output_file_path = config["project_path"] + "/" \
                    + config["temp_folder_path"] + "/" \
                    + config["dist_geo_wt_file"]

dist_res_df.to_excel(output_file_path, header=True, index=False, float_format="%.2f")

# Must Sell

In [ ]:
mustsell_grade_df = pd.merge(must_sell_df, cust_grade_mapping, on=['YearMonth', 'Customer_Code'], how='left')
mustsell_grade_df = mustsell_grade_df.loc[mustsell_grade_df['YearMonth'].isin(['202001', '202002', '202003', '202004', '202005', '202006'])].copy()
mustsell_grade_df['Channel'] = mustsell_grade_df['Channel'].apply(lambda x:get_Channel(x))

In [ ]:
ttl_df = mustsell_grade_df.loc[mustsell_grade_df['Channel']!='Others']
dist_df = mustsell_grade_df.loc[(mustsell_grade_df['Channel']!='Others') & (mustsell_grade_df['Mustsell_All_Distributed']==1)]

In [ ]:
ttl_df[['Mustsell_All_Distributed', 'Channel']].drop_duplicates()

In [ ]:
mustsell_cal_columns = ['YearMonth', 'Geo_Name', 'region_name', 'subregion_name', 'ASM', 'Channel', 'Customer_Code', 'Mustsell_All_Distributed']
mustsell_cal_df = pd.DataFrame(columns=mustsell_cal_columns)

dist_temp = dist_df.groupby(by = ['YearMonth', 'Channel'])['Mustsell_All_Distributed'].sum().reset_index()
dist_temp['Geo_Name'] = '全国'
dist_temp['region_name'] = ''
dist_temp['subregion_name'] = ' '
dist_temp['ASM'] = ''

ttl_temp = ttl_df.groupby(by = ['YearMonth', 'Channel'])['Customer_Code'].count().reset_index()
ttl_temp['Geo_Name'] = '全国'
ttl_temp['region_name'] = ''
ttl_temp['subregion_name'] = ' '
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, dist_temp, on=['YearMonth', 'Channel', 'Geo_Name', 'region_name', 'subregion_name', 'ASM'], how='left')
mustsell_cal_df = mustsell_cal_df.append(df_temp)


dist_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'region_name'])['Mustsell_All_Distributed'].sum().reset_index()
dist_temp['Geo_Name'] = dist_temp['region_name']
dist_temp['subregion_name'] = ' '
dist_temp['ASM'] = ''

ttl_temp = ttl_df.groupby(by = ['YearMonth', 'Channel', 'region_name'])['Customer_Code'].count().reset_index()
ttl_temp['Geo_Name'] = ttl_temp['region_name']
ttl_temp['subregion_name'] = ' '
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, dist_temp, on=['YearMonth', 'Channel', 'Geo_Name', 'region_name', 'subregion_name', 'ASM'], how='left')
mustsell_cal_df = mustsell_cal_df.append(df_temp)


dist_temp = dist_df.groupby(by = ['YearMonth', 'Channel', 'region_name', 'subregion_name'])['Mustsell_All_Distributed'].sum().reset_index()
dist_temp['Geo_Name'] = dist_temp['subregion_name']
dist_temp['ASM'] = ''

ttl_temp = ttl_df.groupby(by = ['YearMonth', 'Channel', 'region_name', 'subregion_name'])['Customer_Code'].count().reset_index()
ttl_temp['Geo_Name'] = ttl_temp['subregion_name']
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, dist_temp, on=['YearMonth', 'Channel', 'Geo_Name', 'region_name', 'subregion_name', 'ASM'], how='left')
mustsell_cal_df = mustsell_cal_df.append(df_temp)

mustsell_cal_df["Customer_Code"] = mustsell_cal_df["Customer_Code"].astype("float")
mustsell_cal_df = mustsell_cal_df.rename(columns={'region_name': 'Region_Name', 
                                                  'subregion_name' : 'Subregion',
                                                  'Customer_Code' : 'Require Must Sell',
                                                  'Mustsell_All_Distributed' : 'Must Sell Stores'})

In [ ]:
mustsell_cal_columns = ['YearMonth', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel', 'Require Must Sell', 'Must Sell Stores']

p3m_df = pd.DataFrame(columns=mustsell_cal_columns)

year = 2020
for month in range(5, 7):
    year_month = str(year) + str(month).zfill(2)
    p3m_list = get_past_3m(year_month)
    
    p3m_sos = mustsell_cal_df.loc[mustsell_cal_df['YearMonth'].isin(p3m_list)].copy()
    mustsell_temp = p3m_sos.groupby(by = ['Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel'])[['Require Must Sell', 'Must Sell Stores']].mean().reset_index()
    mustsell_temp['YearMonth'] = year_month
    
    p3m_df = p3m_df.append(mustsell_temp)
    
p3m_df = p3m_df.rename(columns={'Require Must Sell': 'P3M Require Must Sell', 'Must Sell Stores': 'P3M Must Sell'})
p3m_df['P3M Require Must Sell'] = p3m_df['P3M Require Must Sell'].round(0)
p3m_df['P3M Must Sell'] = p3m_df['P3M Must Sell'].round(0)

In [ ]:
mustsell_res_df = pd.merge(mustsell_cal_df, p3m_df, on=['YearMonth', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel'])

output_file_path = config["project_path"] + "/" \
                    + config["temp_folder_path"] + "/" \
                    + config["mustsell_geo_file"]

mustsell_res_df.to_excel(output_file_path, header=True, index=False, float_format="%.2f")

# Layout

In [ ]:
sfa_cal_df = sfa_input_df
sfa_cal_df['Grade_Group'] = sfa_cal_df['Grade_Code'].apply(lambda x:get_Grade_Group(x))
sfa_cal_df['Channel'] = sfa_cal_df['Grade_Code'].apply(lambda x:get_Channel(x))
sfa_cal_df["Brand"] = sfa_cal_df['SKU'].apply(lambda x:get_brand(x))

sfa_df = sfa_cal_df.loc[(sfa_cal_df['Channel']!='Others') & (sfa_cal_df['Distributed_Store']==1)]

In [ ]:
layout_columns = ['YearMonth', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel', 'Customer_Code', 'Brand' , 'Layout_Store']
layout_df = pd.DataFrame(columns=layout_columns)

# Total Brand
layout_temp = sfa_df.groupby(by = ['YearMonth', 'Channel'])['Layout_Store'].sum().reset_index()
layout_temp['Geo_Name'] = '全国'
layout_temp['Region_Name'] = ''
layout_temp['Subregion'] = ' '
layout_temp['ASM'] = ''

ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Channel'])['Customer_Code'].count().reset_index()
ttl_temp['Geo_Name'] = '全国'
ttl_temp['Region_Name'] = ''
ttl_temp['Subregion'] = ' '
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, layout_temp, on=['YearMonth', 'Channel', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
df_temp['Brand'] = 'TTL'
layout_df = layout_df.append(df_temp)


layout_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Region_Name'])['Layout_Store'].sum().reset_index()
layout_temp['Geo_Name'] = layout_temp['Region_Name']
layout_temp['Subregion'] = ' '
layout_temp['ASM'] = ''

ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Region_Name'])['Customer_Code'].count().reset_index()
ttl_temp['Geo_Name'] = ttl_temp['Region_Name']
ttl_temp['Subregion'] = ' '
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, layout_temp, on=['YearMonth', 'Channel', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
df_temp['Brand'] = 'TTL'
layout_df = layout_df.append(df_temp)


layout_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Region_Name', 'Subregion'])['Layout_Store'].sum().reset_index()
layout_temp['Geo_Name'] = layout_temp['Subregion']
layout_temp['ASM'] = ''

ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Region_Name', 'Subregion'])['Customer_Code'].count().reset_index()
ttl_temp['Geo_Name'] = ttl_temp['Subregion']
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, layout_temp, on=['YearMonth', 'Channel', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
df_temp['Brand'] = 'TTL'
layout_df = layout_df.append(df_temp)


# By Brand
layout_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Brand'])['Layout_Store'].sum().reset_index()
layout_temp['Geo_Name'] = '全国'
layout_temp['Region_Name'] = ''
layout_temp['Subregion'] = ' '
layout_temp['ASM'] = ''

ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Brand'])['Customer_Code'].count().reset_index()
ttl_temp['Geo_Name'] = '全国'
ttl_temp['Region_Name'] = ''
ttl_temp['Subregion'] = ' '
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, layout_temp, on=['YearMonth', 'Channel', 'Brand', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
layout_df = layout_df.append(df_temp)


layout_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Brand', 'Region_Name'])['Layout_Store'].sum().reset_index()
layout_temp['Geo_Name'] = layout_temp['Region_Name']
layout_temp['Subregion'] = ' '
layout_temp['ASM'] = ''

ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Brand', 'Region_Name'])['Customer_Code'].count().reset_index()
ttl_temp['Geo_Name'] = ttl_temp['Region_Name']
ttl_temp['Subregion'] = ' '
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, layout_temp, on=['YearMonth', 'Channel', 'Brand', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
layout_df = layout_df.append(df_temp)


layout_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Brand', 'Region_Name', 'Subregion'])['Layout_Store'].sum().reset_index()
layout_temp['Geo_Name'] = layout_temp['Subregion']
layout_temp['ASM'] = ''

ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Brand', 'Region_Name', 'Subregion'])['Customer_Code'].count().reset_index()
ttl_temp['Geo_Name'] = ttl_temp['Subregion']
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, layout_temp, on=['YearMonth', 'Channel', 'Brand', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
layout_df = layout_df.append(df_temp)

layout_df["Customer_Code"] = layout_df["Customer_Code"].astype("float")
layout_df = layout_df.rename(columns={'Customer_Code' : 'Distributed Stores',
                                    'Layout_Store' : 'Layout Stores'})

In [ ]:
layout_columns = ['YearMonth', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel', 'Brand', 'Distributed Stores', 'Layout Stores']

p3m_df = pd.DataFrame(columns=layout_columns)

year = 2020
for month in range(5, 7):
    year_month = str(year) + str(month).zfill(2)
    p3m_list = get_past_3m(year_month)
    
    p3m_layout = layout_df.loc[layout_df['YearMonth'].isin(p3m_list)].copy()
    layout_temp = p3m_layout.groupby(by = ['Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel', 'Brand'])[['Distributed Stores', 'Layout Stores']].mean().reset_index()
    layout_temp['YearMonth'] = year_month
    
    p3m_df = p3m_df.append(layout_temp)
    
p3m_df = p3m_df.rename(columns={'Distributed Stores': 'P3M Distributed Stores', 'Layout Stores': 'P3M Layout Stores'})
p3m_df['P3M Distributed Stores'] = p3m_df['P3M Distributed Stores'].round(0)
p3m_df['P3M Layout Stores'] = p3m_df['P3M Layout Stores'].round(0)

In [ ]:
layout_res_df = pd.merge(layout_df, p3m_df, on=['YearMonth', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel', 'Brand'])

output_file_path = config["project_path"] + "/" \
                    + config["temp_folder_path"] + "/" \
                    + config["layout_geo_file"]

layout_res_df.to_excel(output_file_path, header=True, index=False, float_format="%.2f")

# Weighted Layout

In [ ]:
sfa_cal_df = sfa_input_df
sfa_cal_df['Grade_Group'] = sfa_cal_df['Grade_Code'].apply(lambda x:get_Grade_Group(x))
sfa_cal_df['Channel'] = sfa_cal_df['Grade_Code'].apply(lambda x:get_Channel(x))
sfa_cal_df["Brand"] = sfa_cal_df['SKU'].apply(lambda x:get_brand(x))

sellout_cust_sku = sellout_with_brand.groupby(by = ['YearMonth', 'Customer_Code', 'Brand_Stage'])[['Sellout_SP_Value']].sum().reset_index()
sellout_cust_sku.columns = ['YearMonth', 'Customer_Code', 'SKU', 'Sellout_SP_Value']

sfa_cal_df = pd.merge(sfa_cal_df, sellout_cust_sku, on=['YearMonth', 'Customer_Code'], how='left')
sfa_cal_df = sfa_cal_df.fillna({'Sellout_SP_Value':0})
sfa_cal_df['Layout_Store'] = sfa_cal_df['Layout_Store'] * sfa_cal_df['Sellout_SP_Value']

sfa_df = sfa_cal_df.loc[(sfa_cal_df['Channel']!='Others') & (sfa_cal_df['Distributed_Store']==1)]

In [ ]:
layout_columns = ['YearMonth', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel', 'Sellout_SP_Value', 'Brand' , 'Layout_Store']
layout_df = pd.DataFrame(columns=layout_columns)

# Total Brand
layout_temp = sfa_df.groupby(by = ['YearMonth', 'Channel'])['Layout_Store'].sum().reset_index()
layout_temp['Geo_Name'] = '全国'
layout_temp['Region_Name'] = ''
layout_temp['Subregion'] = ' '
layout_temp['ASM'] = ''

ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Channel'])['Sellout_SP_Value'].sum().reset_index()
ttl_temp['Geo_Name'] = '全国'
ttl_temp['Region_Name'] = ''
ttl_temp['Subregion'] = ' '
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, layout_temp, on=['YearMonth', 'Channel', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
df_temp['Brand'] = 'TTL'
layout_df = layout_df.append(df_temp)


layout_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Region_Name'])['Layout_Store'].sum().reset_index()
layout_temp['Geo_Name'] = layout_temp['Region_Name']
layout_temp['Subregion'] = ' '
layout_temp['ASM'] = ''

ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Region_Name'])['Sellout_SP_Value'].sum().reset_index()
ttl_temp['Geo_Name'] = ttl_temp['Region_Name']
ttl_temp['Subregion'] = ' '
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, layout_temp, on=['YearMonth', 'Channel', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
df_temp['Brand'] = 'TTL'
layout_df = layout_df.append(df_temp)


layout_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Region_Name', 'Subregion'])['Layout_Store'].sum().reset_index()
layout_temp['Geo_Name'] = layout_temp['Subregion']
layout_temp['ASM'] = ''

ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Region_Name', 'Subregion'])['Sellout_SP_Value'].sum().reset_index()
ttl_temp['Geo_Name'] = ttl_temp['Subregion']
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, layout_temp, on=['YearMonth', 'Channel', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
df_temp['Brand'] = 'TTL'
layout_df = layout_df.append(df_temp)


# By Brand
layout_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Brand'])['Layout_Store'].sum().reset_index()
layout_temp['Geo_Name'] = '全国'
layout_temp['Region_Name'] = ''
layout_temp['Subregion'] = ' '
layout_temp['ASM'] = ''

ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Brand'])['Sellout_SP_Value'].sum().reset_index()
ttl_temp['Geo_Name'] = '全国'
ttl_temp['Region_Name'] = ''
ttl_temp['Subregion'] = ' '
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, layout_temp, on=['YearMonth', 'Channel', 'Brand', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
layout_df = layout_df.append(df_temp)


layout_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Brand', 'Region_Name'])['Layout_Store'].sum().reset_index()
layout_temp['Geo_Name'] = layout_temp['Region_Name']
layout_temp['Subregion'] = ' '
layout_temp['ASM'] = ''

ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Brand', 'Region_Name'])['Sellout_SP_Value'].sum().reset_index()
ttl_temp['Geo_Name'] = ttl_temp['Region_Name']
ttl_temp['Subregion'] = ' '
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, layout_temp, on=['YearMonth', 'Channel', 'Brand', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
layout_df = layout_df.append(df_temp)


layout_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Brand', 'Region_Name', 'Subregion'])['Layout_Store'].sum().reset_index()
layout_temp['Geo_Name'] = layout_temp['Subregion']
layout_temp['ASM'] = ''

ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Channel', 'Brand', 'Region_Name', 'Subregion'])['Sellout_SP_Value'].sum().reset_index()
ttl_temp['Geo_Name'] = ttl_temp['Subregion']
ttl_temp['ASM'] = ''

df_temp = pd.merge(ttl_temp, layout_temp, on=['YearMonth', 'Channel', 'Brand', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM'], how='left')
layout_df = layout_df.append(df_temp)

layout_df["Sellout_SP_Value"] = layout_df["Sellout_SP_Value"].astype("float")
layout_df = layout_df.rename(columns={'Sellout_SP_Value' : 'Weighted Distributed Stores',
                                    'Layout_Store' : 'Weighted Layout Stores'})

In [ ]:
layout_columns = ['YearMonth', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel', 'Brand', 'Weighted Distributed Stores', 'Weighted Layout Stores']

p3m_df = pd.DataFrame(columns=layout_columns)

year = 2020
for month in range(5, 7):
    year_month = str(year) + str(month).zfill(2)
    p3m_list = get_past_3m(year_month)
    
    p3m_layout = layout_df.loc[layout_df['YearMonth'].isin(p3m_list)].copy()
    layout_temp = p3m_layout.groupby(by = ['Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel', 'Brand'])[['Weighted Distributed Stores', 'Weighted Layout Stores']].mean().reset_index()
    layout_temp['YearMonth'] = year_month
    
    p3m_df = p3m_df.append(layout_temp)
    
p3m_df = p3m_df.rename(columns={'Weighted Distributed Stores': 'P3M Weighted Distributed', 'Weighted Layout Stores': 'P3M Weighted Layout'})

In [ ]:
layout_res_df = pd.merge(layout_df, p3m_df, on=['YearMonth', 'Geo_Name', 'Region_Name', 'Subregion', 'ASM', 'Channel', 'Brand'])

output_file_path = config["project_path"] + "/" \
                    + config["temp_folder_path"] + "/" \
                    + config["layout_geo_wt_file"]

layout_res_df.to_excel(output_file_path, header=True, index=False, float_format="%.2f")

# OOS 

In [ ]:
sfa_cal_df = sfa_input_df
sfa_cal_df['Channel'] = sfa_cal_df['Channel'].apply(lambda x:get_Channel(x))
sfa_cal_df["Brand"] = sfa_cal_df['SKU'].apply(lambda x:get_brand(x))

sfa_cal_df['Grade_Group'] = sfa_cal_df['Grade_Code'].apply(lambda x:get_Grade_Group(x))

In [ ]:
sfa_df = sfa_cal_df.loc[(sfa_cal_df['Channel']!='Others') & (sfa_cal_df['Distributed_Store']==1)]

In [ ]:
oos_columns = ['YearMonth', 'Grade_Group', 'Account_Name', 'Brand', 'SKU', 'Total_Stores', 'Shortage_Store']

oos_df = pd.DataFrame(columns=oos_columns)

oos_temp = sfa_df.groupby(by = ['YearMonth'])['Shortage_Store'].sum().reset_index()
oos_temp['Account_Name'] = ''
oos_temp['Grade_Group'] = 'Offline'
oos_temp['Brand'] = 'ELN DC'
oos_temp['SKU'] = 'ELN DC'

ttl_temp = sfa_df.groupby(by = ['YearMonth'])['Customer_Code'].count().reset_index()
ttl_temp['Account_Name'] = ''
ttl_temp['Grade_Group'] = 'Offline'
ttl_temp['Brand'] = 'ELN DC'
ttl_temp['SKU'] = 'ELN DC'

df_temp = pd.merge(ttl_temp, oos_temp, on=['YearMonth', 'Grade_Group', 'Account_Name', 'Brand', 'SKU'], how='left')
df_temp = df_temp.rename(columns={'Customer_Code': 'Total_Stores'})
oos_df = oos_df.append(df_temp)


oos_temp = sfa_df.groupby(by = ['YearMonth', 'Brand'])['Shortage_Store'].sum().reset_index()
oos_temp['Account_Name'] = ''
oos_temp['Grade_Group'] = 'Offline'
oos_temp['SKU'] = oos_temp['Brand']
oos_temp['Brand'] = 'ELN DC'

ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Brand'])['Customer_Code'].count().reset_index()
ttl_temp['Account_Name'] = ''
ttl_temp['Grade_Group'] = 'Offline'
ttl_temp['SKU'] =ttl_temp['Brand']
ttl_temp['Brand'] = 'ELN DC'

df_temp = pd.merge(ttl_temp, oos_temp, on=['YearMonth', 'Grade_Group', 'Account_Name', 'Brand', 'SKU'], how='left')
df_temp = df_temp.rename(columns={'Customer_Code': 'Total_Stores'})
oos_df = oos_df.append(df_temp)


oos_temp = sfa_df.groupby(by = ['YearMonth', 'Brand', 'SKU'])['Shortage_Store'].sum().reset_index()
oos_temp['Account_Name'] = ''
oos_temp['Grade_Group'] = 'Offline'
ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Brand', 'SKU'])['Customer_Code'].count().reset_index()
ttl_temp['Account_Name'] = ''
ttl_temp['Grade_Group'] = 'Offline'

df_temp = pd.merge(ttl_temp, oos_temp, on=['YearMonth', 'Grade_Group', 'Account_Name', 'Brand', 'SKU'], how='left')
df_temp = df_temp.rename(columns={'Customer_Code': 'Total_Stores'})
oos_df = oos_df.append(df_temp)

In [ ]:
oos_temp = sfa_df.groupby(by = ['YearMonth', 'Grade_Group'])['Shortage_Store'].sum().reset_index()
oos_temp['Account_Name'] = ''
oos_temp['Brand'] = 'ELN DC'
oos_temp['SKU'] = 'ELN DC'

ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Grade_Group'])['Customer_Code'].count().reset_index()
ttl_temp['Account_Name'] = ''
ttl_temp['Brand'] = 'ELN DC'
ttl_temp['SKU'] = 'ELN DC'

df_temp = pd.merge(ttl_temp, oos_temp, on=['YearMonth', 'Grade_Group', 'Account_Name', 'Brand', 'SKU'], how='left')
df_temp = df_temp.rename(columns={'Customer_Code': 'Total_Stores'})
oos_df = oos_df.append(df_temp)


oos_temp = sfa_df.groupby(by = ['YearMonth', 'Grade_Group', 'Brand'])['Shortage_Store'].sum().reset_index()
oos_temp['Account_Name'] = ''
oos_temp['SKU'] = oos_temp['Brand']
oos_temp['Brand'] = 'ELN DC'

ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Grade_Group', 'Brand'])['Customer_Code'].count().reset_index()
ttl_temp['Account_Name'] = ''
ttl_temp['SKU'] =ttl_temp['Brand']
ttl_temp['Brand'] = 'ELN DC'

df_temp = pd.merge(ttl_temp, oos_temp, on=['YearMonth', 'Grade_Group', 'Account_Name', 'Brand', 'SKU'], how='left')
df_temp = df_temp.rename(columns={'Customer_Code': 'Total_Stores'})
oos_df = oos_df.append(df_temp)


oos_temp = sfa_df.groupby(by = ['YearMonth', 'Grade_Group', 'Brand', 'SKU'])['Shortage_Store'].sum().reset_index()
oos_temp['Account_Name'] = ''
ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Grade_Group', 'Brand', 'SKU'])['Customer_Code'].count().reset_index()
ttl_temp['Account_Name'] = ''

df_temp = pd.merge(ttl_temp, oos_temp, on=['YearMonth', 'Grade_Group', 'Account_Name', 'Brand', 'SKU'], how='left')
df_temp = df_temp.rename(columns={'Customer_Code': 'Total_Stores'})
oos_df = oos_df.append(df_temp)

In [ ]:
oos_temp = sfa_df.groupby(by = ['YearMonth', 'Grade_Group', 'Account_Name'])['Shortage_Store'].sum().reset_index()
oos_temp['Brand'] = 'ELN DC'
oos_temp['SKU'] = 'ELN DC'

ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Grade_Group', 'Account_Name'])['Customer_Code'].count().reset_index()
ttl_temp['Brand'] = 'ELN DC'
ttl_temp['SKU'] = 'ELN DC'

df_temp = pd.merge(ttl_temp, oos_temp, on=['YearMonth', 'Grade_Group', 'Account_Name', 'Brand', 'SKU'], how='left')
df_temp = df_temp.rename(columns={'Customer_Code': 'Total_Stores'})
oos_df = oos_df.append(df_temp)


oos_temp = sfa_df.groupby(by = ['YearMonth', 'Grade_Group', 'Account_Name', 'Brand'])['Shortage_Store'].sum().reset_index()
oos_temp['SKU'] = oos_temp['Brand']
oos_temp['Brand'] = 'ELN DC'

ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Grade_Group', 'Account_Name', 'Brand'])['Customer_Code'].count().reset_index()
ttl_temp['SKU'] =ttl_temp['Brand']
ttl_temp['Brand'] = 'ELN DC'

df_temp = pd.merge(ttl_temp, oos_temp, on=['YearMonth', 'Grade_Group', 'Account_Name', 'Brand', 'SKU'], how='left')
df_temp = df_temp.rename(columns={'Customer_Code': 'Total_Stores'})
oos_df = oos_df.append(df_temp)


oos_temp = sfa_df.groupby(by = ['YearMonth', 'Grade_Group', 'Account_Name', 'Brand', 'SKU'])['Shortage_Store'].sum().reset_index()
ttl_temp = sfa_df.groupby(by = ['YearMonth', 'Grade_Group', 'Account_Name', 'Brand', 'SKU'])['Customer_Code'].count().reset_index()

df_temp = pd.merge(ttl_temp, oos_temp, on=['YearMonth', 'Grade_Group', 'Account_Name', 'Brand', 'SKU'], how='left')
df_temp = df_temp.rename(columns={'Customer_Code': 'Total_Stores'})
oos_df = oos_df.append(df_temp)

In [ ]:
oos_df['OOS_Rate'] = oos_df['Shortage_Store'] / oos_df['Total_Stores']

In [ ]:
output_file_path = config["project_path"] + "/" \
                    + config["temp_folder_path"] + "/" \
                    + config["OOS_grade_file"]

oos_df.to_excel(output_file_path, header=True, index=False, float_format="%.2f")